# 同心云个人信息爬虫

### 基于http://yun.tongji.edu.cn/addressbook/home?pageIndex= 提供的名单（含用户ID）分别在 http://yun.tongji.edu.cn/microblog/…/profile 和 http://yun.tongji.edu.cn/microblog/rest/microCard/getByUserId?userId=  获取个人[身份/学历]、[学院]、[专业]、[年级]、[邮箱]和[姓名]、[电话]。

- 该版本运行后提供了可编辑的DataFrame [mydata] ,也同时实现了本地.csv保存功能。该DataFrame并不实时写出，而是只在读取完所有信息后写一次.csv，因而相当耗费内存。另一个即读即存的版本详见personal_information_pro。
- 提供了文本形式的进度条。
- 对于mydata可以调用如下参数（默认都是str格式）：

字段参数 | 含义 | 备注 
-|-|-
degree | 在读学历 | ["本科生","研究生","博士生","教职工"] |
school | 学院 | / |
major | 专业 | 对于身份为"教职工"的用户，该参数的值为"" |
year | 年级 | / |
mail | 邮箱 | 该参数可能缺省为"" |
name | 姓名 | / |
mobile | 电话 | 该参数可能缺省为"" |

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import numpy as npy
import re
import json
url = "http://yun.tongji.edu.cn/space/c/rest/user/login"
UA = "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.13 Safari/537.36"
 
header = { "User-Agent" : UA,
           "Referer": "http://yun.tongji.edu.cn/public/login?supportNotMobile=false"
           }
 
txy_session = requests.Session()
f = txy_session.get(url,headers=header)
 
myphone=input("同心云登陆手机号:")
psw=input("同心云登录密码:")

postData = {
    'email':myphone,
    'password': psw,
    'remember':'true',
    'accountType':'',
    'accountName':'',
    'networkId':'',
    'redirectUrl':'', 
    'forceToNetwork':'false'
}
 
txy_session.post(url,
                  data = postData,
                  headers = header)

# print(f.content.decode())



pattern1="(?<=data-userid=\").*(?=\" data-username)"  #在addressbook页面匹配所有data-userid后面接的内容
pattern2="(?<=部门：\n					<span title=\").*(?=\">)"  #在/profile页面匹配所有部门。注意，部门这个参数拆解后不定长，可能是4或5
pattern3="(?<=\n<li>邮箱：<a href=\"mailto:).*(?=\">)"  #在/profile页面匹配所有邮箱。虽然长得很难看但却是能准确匹配到。

degree=[]
school=[]
major=[]
year=[]
mail=[]
user_name=[]
mobile=[]
mail2=[]

lower_bound=input("输入页数下界（将包含下界）（min=1）")
upper_bound=input("输入页数上界 (将不含上界) (max=2913) ")
filename=input("生成.csv的名字：")
num_of_pages=int(int(upper_bound)-int(lower_bound))
page_count=-1
print("将处理 "+ str(num_of_pages) +" 页共 "+str(num_of_pages*20)+" 条数据\n")
for items in range(int(lower_bound),int(upper_bound)):
# for items in range(1,3):
    page_count+=1
    src_1="http://yun.tongji.edu.cn/addressbook/home?pageIndex="+str(items)
    print(src_1)
    f = txy_session.get(src_1,headers=header)
    soup = BeautifulSoup(f.content,"html.parser")
    curr_content=str(soup.find('tbody'))
    pt1=re.findall(pattern1,curr_content)
    perc=0
    for i in pt1:
        perc+=5
        print("\rProcessing "+str(perc)+" % on "+str(page_count)+" of "+str(num_of_pages)+" pages...",end=" ")
        #进入/profile页面(只含一个人的信息):
        src_2 = "http://yun.tongji.edu.cn/microblog/"+ i +"/profile"
        #print(src_2)
        ff=txy_session.get(src_2,headers=header)
        soup2 = BeautifulSoup(ff.content,"html.parser")
        curr_content2=str(soup2.find('div',{'class':'bd c1'})) #缩小查找范围(可能没必要？)
        #print(curr_content2)
        #第一部分：获取专业和年级信息
        pt2=re.findall(pattern2,curr_content2)
        pt2=str(pt2)
        curr_content3=pt2.split("&gt;")
        try:
            #print(curr_content3[1])
            degree.append(curr_content3[1])
        except Exception as ee:
            degree.append("") # never happen
        try:
            #print(curr_content3[2])
            school.append(curr_content3[2])
        except Exception as ee:
            school.append("") # never happen
        try:  #对于专业有必要做分类讨论，教职工没有该属性
            if(len(curr_content3)>4):
                #print(curr_content3[3])
                major.append(curr_content3[3])
            else:
                #print("")
                major.append("")
        except Exception as ee:
            major.append("")  
        try:
            #print(curr_content3[len(curr_content3)-1][0:4])
            year.append(curr_content3[len(curr_content3)-1][0:4])
        except Exception as ee:
            year.append("") # last element alaways = year
            
        #第二部分：获取邮箱 (仍沿用curr_content2)/ 如果是qq邮箱解析出qq号
        pt3=str(re.findall(pattern3,curr_content2))
        if(pt3!=''):
            pt3=pt3[2:len(pt3)-2]  #我也不太明白为什么原生数据会带一对双引号和方括号
            mail.append(pt3)
            #print(pt3)
        else:
            mail.append("")
            #print("")
        
        #其实可以用下面部分的email参数来try的，一时没考虑到。
        
        #第三部分：获取个人姓名和手机号（一定成功 不用try）以及更多信息(需要try)
        src_3="http://yun.tongji.edu.cn/microblog/rest/microCard/getByUserId?userId="+i
        #print(src3)
        fff=txy_session.get(src_3,headers=header)
        soup3 = BeautifulSoup(fff.content,"html.parser")
        s = json.loads(str(soup3))           
        user_name.append(str(s['microCard']['name']))
        try:
            mobile.append(str(s['microCard']['mobiles'][0]))
        except Exception as ee:
            mobile.append("")
#         try:  
#             mail2.append(str(s['microCard']['email']))
#         except:
#             mail2.append("")


summary={"name":user_name,"degree":degree,"school":school,"major":major,"year":year,"mail":mail,"mobile":mobile}
mydata=pd.DataFrame(summary)
print("\r完成！                                                ",end=" ")
mydata.to_csv(filename+".csv", mode='a', index=False, sep=',', header=False, encoding="utf_8_sig" )
